In [19]:
path_to_fix = r"C:\_projekty_repo\noc\thermal_game\data\week01_prices_weather_seasons_2025-01-01_old copy 2.csv"

df_what_we_want = pd.read_csv(path_to_fix)
df_what_we_want

,timestamp,t,day,dt_h,hour_of_day,month,season,dayofyear,t_out_c,price_eur_per_kwh,solar_gen_kw_per_kwp,in_work_hours
0,2025-01-01 00:00:00,0,1,0.25,0.00,1,winter,1,-1.9350,0.07583,0.0,0
1,2025-01-02 00:00:00,1,1,0.25,0.25,1,winter,1,-2.0600,0.06316,0.0,0
2,2025-01-03 00:00:00,2,1,0.25,0.50,1,winter,1,-2.1825,0.05465,0.0,0
3,2025-01-04 00:00:00,3,1,0.25,0.75,1,winter,1,-2.3050,0.03550,0.0,0
4,2025-01-05 00:00:00,4,1,0.25,1.00,1,winter,1,-2.4275,0.06466,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
667,2026-10-30 00:00:00,667,7,0.25,22.75,1,winter,7,-6.2650,0.04774,0.0,0
668,2026-10-31 00:00:00,668,7,0.25,23.00,1,winter,7,-6.3225,0.03223,0.0,0
669,2026-11-01 00:00:00,669,7,0.25,23.25,1,winter,7,-6.7700,0.06550,0.0,0
670,2026-11-02 00:00:00,670,7,0.25,23.50,1,winter,7,-6.9000,0.05512,0.0,0


In [2]:
import pandas as pd
#headers:timestamp,t,day,dt_h,hour_of_day,month,season,dayofyear,t_out_c,price_eur_per_kwh,solar_gen_kw_per_kwp,in_work_hours
path_1 = r"C:\_projekty_repo\noc\thermal_game\data\prices_15min.csv"
path2 = r"C:\_projekty_repo\noc\thermal_game\data\weather_15min.csv"

df_1 = pd.read_csv(path_1,index_col=0,)
df_2 = pd.read_csv(path2, index_col=0,)


In [21]:
import pandas as pd
import numpy as np

# --- inputs you already have ---
# path_1 = r"C:\_projekty_repo\noc\thermal_game\data\prices_15min.csv"
# path2  = r"C:\_projekty_repo\noc\thermal_game\data\weather_15min.csv"
# df_1 = pd.read_csv(path_1, index_col=0)
# df_2 = pd.read_csv(path2,  index_col=0)

# ---------- 0) Ensure datetime index, tidy ----------
for _df in (df_1, df_2):
    if not isinstance(_df.index, pd.DatetimeIndex):
        _df.index = pd.to_datetime(_df.index, errors="coerce", utc=False)
    _df.sort_index(inplace=True)
    _df.rename(columns=str.strip, inplace=True)

# ---------- 1) Select year 2023 ----------
p23 = df_1[df_1.index.year == 2023].copy()
w23 = df_2[df_2.index.year == 2023].copy()

if p23.empty or w23.empty:
    raise ValueError("No 2023 data found in one or both sources (prices/weather).")

# ---------- 2) Change year from 2023 -> 2025 (keep month/day/time) ----------
# (2023 and 2025 are both non-leap years, so this is safe.)
p25 = p23.copy()
w25 = w23.copy()
p25.index = p25.index.map(lambda ts: ts.replace(year=2025))
w25.index = w25.index.map(lambda ts: ts.replace(year=2025))

# Optional: if you might collide with existing 2025 rows, keep the 2023->2025 relabeled data
# p25 = p25[~p25.index.isin(df_1[df_1.index.year == 2025].index)]
# w25 = w25[~w25.index.isin(df_2[df_2.index.year == 2025].index)]

# ---------- 3) Join on the relabeled 2025 timestamps ----------
df = p25.join(w25, how="inner")
df.index.name = "timestamp"
if df.empty:
    raise ValueError("After relabeling to 2025, there are no overlapping timestamps between price and weather.")

# ---------- 4) Derive requested features ----------
idx = df.index

# dt_h: detect step, default 0.25 (15 min)
dt_series = idx.to_series().diff().dt.total_seconds().div(3600)
step = dt_series.mode().iloc[0] if not dt_series.mode().empty else dt_series.median()
if not np.isfinite(step) or step <= 0:
    step = 0.25
dt_h = dt_series.fillna(step)

# hour_of_day
hour_of_day = pd.Series(idx.hour + idx.minute/60 + idx.second/3600, index=idx, name="hour_of_day")

# month -> season
def month_to_season(m: int) -> str:
    if m in (12, 1, 2): return "winter"
    if m in (3, 4, 5):  return "spring"
    if m in (6, 7, 8):  return "summer"
    return "autumn"
season = pd.Series([month_to_season(m) for m in idx.month], index=idx, name="season")

# t_out_c (primary: ambient_temp_C; fallback common names)
if "ambient_temp_C" in df.columns:
    t_out_c = df["ambient_temp_C"]
else:
    cand = [c for c in df.columns if c.lower() in ("t_out_c","t_out","temp_c","temperature_c")]
    t_out_c = df[cand[0]] if cand else pd.Series(np.nan, index=idx, name="t_out_c")

# price_eur_per_kwh from elec_price_EUR_per_MWh
if "elec_price_EUR_per_MWh" in df.columns:
    price_eur_per_kwh = df["elec_price_EUR_per_MWh"] / 1000.0
else:
    cand = [c for c in df.columns if ("price" in c.lower()) and ("mwh" in c.lower())]
    price_eur_per_kwh = (df[cand[0]] / 1000.0) if cand else pd.Series(np.nan, index=idx, name="price_eur_per_kwh")

# solar_gen_kw_per_kwp: irradiance proxy
if "irradiance_Wm2" in df.columns:
    solar_gen_kw_per_kwp = (df["irradiance_Wm2"].clip(lower=0) / 1000.0).fillna(0.0)
elif "P[W]" in df.columns:
    solar_gen_kw_per_kwp = (df["P[W]"].clip(lower=0) / 1000.0)
else:
    solar_gen_kw_per_kwp = pd.Series(0.0, index=idx, name="solar_gen_kw_per_kwp")

# in_work_hours (Mon–Fri, 09:00–17:00)
BUSINESS_START = 9
BUSINESS_END   = 17
in_work_hours = pd.Series(
    ((idx.dayofweek < 5) & (idx.hour >= BUSINESS_START) & (idx.hour < BUSINESS_END)).astype(int),
    index=idx, name="in_work_hours"
)

# day as INT (YYYYMMDD) — and t as sequential step
day_int = (idx.year * 10000 + idx.month * 100 + idx.day).astype(int)
t_seq = np.arange(len(df), dtype=int)

# ---------- 5) Build final dataframe with exact header order ----------
df_what_we_want = pd.DataFrame({
    "timestamp": idx,                                   # keep as column
    "t": t_seq,
    "day": day_int,                                     # INT
    "dt_h": pd.to_numeric(dt_h, errors="coerce"),
    "hour_of_day": pd.to_numeric(hour_of_day, errors="coerce"),
    "month": idx.month.astype(int),
    "season": season.astype(str),
    "dayofyear": idx.dayofyear.astype(int),
    "t_out_c": pd.to_numeric(t_out_c, errors="coerce"),
    "price_eur_per_kwh": pd.to_numeric(price_eur_per_kwh, errors="coerce"),
    "solar_gen_kw_per_kwp": pd.to_numeric(solar_gen_kw_per_kwp, errors="coerce"),
    "in_work_hours": in_work_hours.astype(int)
}, index=idx)[[
    "timestamp","t","day","dt_h","hour_of_day","month","season","dayofyear",
    "t_out_c","price_eur_per_kwh","solar_gen_kw_per_kwp","in_work_hours"
]]

# Optional: keep only column form (like your CSVs)
df_what_we_want = df_what_we_want.reset_index(drop=True)

# ---------- 6) Preview ----------
print(df_what_we_want.head())
print(df_what_we_want.dtypes)

# Optional: save
out_path = r"C:\_projekty_repo\noc\thermal_game\data\weekXX_prices_weather_seasons_FROM_2023_RELABELED_TO_2025.csv"
df_what_we_want.to_csv(out_path, index=False)


            timestamp  t       day  dt_h  hour_of_day  month  season  \
0 2025-01-01 00:00:00  0  20250101  0.25         0.00      1  winter   
1 2025-01-01 00:15:00  1  20250101  0.25         0.25      1  winter   
2 2025-01-01 00:30:00  2  20250101  0.25         0.50      1  winter   
3 2025-01-01 00:45:00  3  20250101  0.25         0.75      1  winter   
4 2025-01-01 01:00:00  4  20250101  0.25         1.00      1  winter   

   dayofyear  t_out_c  price_eur_per_kwh  solar_gen_kw_per_kwp  in_work_hours  
0          1   8.2825            0.07501                   0.0              0  
1          1   8.2200           -0.02995                   0.0              0  
2          1   8.2025           -0.02998                   0.0              0  
3          1   8.1850           -0.03000                   0.0              0  
4          1   8.1675            0.02681                   0.0              0  
timestamp               datetime64[ns]
t                                int64
day      

In [8]:
import pandas as pd
import numpy as np

# --- load ---
# df_1 = pd.read_csv(path_1)   # prices
# df_2 = pd.read_csv(path_2)   # weather

def _to_dt_index(df, ts_col=None):
    if ts_col is not None:
        ts = pd.to_datetime(df[ts_col], errors="coerce", utc=True).dt.tz_convert(None)
        df = df.drop(columns=[ts_col])
        df.index = ts
    elif not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, errors="coerce", utc=True).tz_convert(None)
    else:
        if df.index.tz is not None:
            df.index = df.index.tz_convert(None)
    df.sort_index(inplace=True)
    return df

def _snap_15min(idx):
    return idx.round("15min")

def _dedupe_keep_first(df):
    return df[~df.index.duplicated(keep="first")]

# ---------- 0) Normalize, snap, dedupe ----------
df_1 = _to_dt_index(df_1)
df_2 = _to_dt_index(df_2)

df_1.index = _snap_15min(df_1.index)
df_2.index = _snap_15min(df_2.index)

df_1 = _dedupe_keep_first(df_1)
df_2 = _dedupe_keep_first(df_2)

# ---------- 1) Restrict to 2023 & canonical grid ----------
start_2023 = pd.Timestamp("2023-01-01 00:00:00")
end_2023   = pd.Timestamp("2023-12-31 23:45:00")
grid_2023  = pd.date_range(start_2023, end_2023, freq="15min")

p23 = df_1.loc[(df_1.index >= start_2023) & (df_1.index <= end_2023)].copy()
w23 = df_2.loc[(df_2.index >= start_2023) & (df_2.index <= end_2023)].copy()

p23 = p23.reindex(grid_2023)
w23 = w23.reindex(grid_2023)

# ---------- fill missing 2023 PRICES from 2025 or 2021 ----------
def _find_price_col(df):
    for c in df.columns:
        cl = c.lower()
        if "elec" in cl and "price" in cl and "mwh" in cl:
            return c
    raise ValueError("Price column like 'elec_price_EUR_per_MWh' not found.")

price_col = _find_price_col(p23)

def donor_fill_by_md_hm(target_series, donor_df, donor_year, donor_col):
    """Fill NaNs in target_series using donor_df values from donor_year, matching by (month, day, hour, minute)."""
    if donor_df.empty:
        return target_series

    ddf = donor_df[donor_df.index.year == donor_year]
    if ddf.empty:
        return target_series

    # Build mapping (month, day, hour, minute) -> value
    donor_key = pd.MultiIndex.from_arrays(
        [ddf.index.month, ddf.index.day, ddf.index.hour, ddf.index.minute]
    )
    donor_map = pd.Series(ddf[donor_col].values, index=donor_key)

    miss_idx = target_series.index[target_series.isna()]
    if len(miss_idx) == 0:
        return target_series

    # Build keys for missing timestamps
    miss_keys = pd.MultiIndex.from_arrays(
        [miss_idx.month, miss_idx.day, miss_idx.hour, miss_idx.minute]
    )

    # Align donor values to missing timestamps
    repl = pd.Series(donor_map.reindex(miss_keys).values, index=miss_idx)

    out = target_series.copy()
    out.loc[miss_idx] = out.loc[miss_idx].fillna(repl)
    return out


# try donor 2025, then 2021
p23[price_col] = donor_fill_by_md_hm(p23[price_col], df_1, 2025, price_col)
p23[price_col] = donor_fill_by_md_hm(p23[price_col], df_1, 2021, price_col)

# small-gap interpolation (≤96 steps = 1 day)
p23[price_col] = p23[price_col].astype(float).interpolate(method="time", limit=96)

# safety net
p23[price_col] = p23[price_col].ffill().bfill()

# --- smooth with moving average of 4 steps (1 hour window on 15-min data) ---
p23[price_col] = p23[price_col].rolling(window=18, min_periods=1).mean()

# weather gaps: interpolate small runs, ffill/bfill
for c in w23.columns:
    if w23[c].dtype.kind in "biufc":
        w23[c] = w23[c].interpolate(method="time", limit=8).ffill().bfill()

# optional: final NaN check
if p23[price_col].isna().any() or w23.isna().any().any():
    bad_p = int(p23[price_col].isna().sum())
    bad_w = int(w23.isna().sum().sum())
    raise ValueError(f"Still have NaNs after fills — price:{bad_p}, weather:{bad_w}")

# ---------- 2) Relabel to 2025 ----------
p25 = p23.copy()
w25 = w23.copy()
p25.index = p25.index + pd.DateOffset(years=2)
w25.index = w25.index + pd.DateOffset(years=2)

# ---------- 3) Join ----------
df = p25.join(w25, how="inner")
df.index.name = "timestamp"

# ---------- 4) Features ----------
idx = df.index
hour_of_day = idx.hour + idx.minute/60 + idx.second/3600
def month_to_season(m: int) -> str:
    return ("winter" if m in (12,1,2) else
            "spring" if m in (3,4,5) else
            "summer" if m in (6,7,8) else
            "autumn")
season = pd.Series([month_to_season(m) for m in idx.month], index=idx)

if "ambient_temp_C" in df.columns:
    t_out_c = df["ambient_temp_C"]
else:
    cand = [c for c in df.columns if c.lower() in ("t_out_c","t_out","temp_c","temperature_c")]
    t_out_c = df[cand[0]] if cand else pd.Series(np.nan, index=idx)

if "elec_price_EUR_per_MWh" in df.columns:
    price_eur_per_kwh = df["elec_price_EUR_per_MWh"] / 1000.0
else:
    cand = [c for c in df.columns if ("price" in c.lower()) and ("mwh" in c.lower())]
    price_eur_per_kwh = (df[cand[0]] / 1000.0) if cand else pd.Series(np.nan, index=idx)

if "irradiance_Wm2" in df.columns:
    solar_gen_kw_per_kwp = (df["irradiance_Wm2"].clip(lower=0) / 1000.0).fillna(0.0)
elif "P[W]" in df.columns:
    solar_gen_kw_per_kwp = (df["P[W]"].clip(lower=0) / 1000.0)
else:
    solar_gen_kw_per_kwp = pd.Series(0.0, index=idx)

BUSINESS_START, BUSINESS_END = 9, 17
in_work_hours = ((idx.dayofweek < 5) & (idx.hour >= BUSINESS_START) & (idx.hour < BUSINESS_END)).astype(int)

day_int = (idx.year * 10000 + idx.month * 100 + idx.day).astype(int)
t_seq = np.arange(len(df), dtype=int)

df_out = pd.DataFrame({
    "timestamp": idx,
    "t": t_seq,
    "day": day_int,
    "dt_h": 0.25,
    "hour_of_day": hour_of_day,
    "month": idx.month.astype(int),
    "season": season.astype(str),
    "dayofyear": idx.dayofyear.astype(int),
    "t_out_c": pd.to_numeric(t_out_c, errors="coerce"),
    "price_eur_per_kwh": pd.to_numeric(price_eur_per_kwh, errors="coerce"),
    "solar_gen_kw_per_kwp": pd.to_numeric(solar_gen_kw_per_kwp, errors="coerce"),
    "in_work_hours": in_work_hours.astype(int),
}, index=idx)[[
    "timestamp","t","day","dt_h","hour_of_day","month","season","dayofyear",
    "t_out_c","price_eur_per_kwh","solar_gen_kw_per_kwp","in_work_hours"
]].reset_index(drop=True)

print(df_out.head(), df_out.dtypes)

out_path = r"C:\_projekty_repo\noc\thermal_game\data\_2ndweekXX_prices_weather_seasons_FROM_2023_RELABELED_TO_2025.csv"
df_out.to_csv(out_path, index=False)

            timestamp  t       day  dt_h  hour_of_day  month  season  \
0 2025-01-01 00:00:00  0  20250101  0.25         0.00      1  winter   
1 2025-01-01 00:15:00  1  20250101  0.25         0.25      1  winter   
2 2025-01-01 00:30:00  2  20250101  0.25         0.50      1  winter   
3 2025-01-01 00:45:00  3  20250101  0.25         0.75      1  winter   
4 2025-01-01 01:00:00  4  20250101  0.25         1.00      1  winter   

   dayofyear  t_out_c  price_eur_per_kwh  solar_gen_kw_per_kwp  in_work_hours  
0          1   8.2825           0.075010                   0.0              0  
1          1   8.2200           0.022530                   0.0              0  
2          1   8.2025           0.005027                   0.0              0  
3          1   8.1850          -0.003730                   0.0              0  
4          1   8.1675           0.002378                   0.0              0   timestamp               datetime64[ns]
t                                int64
day      